In [1]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import DataLoader
from train import FastflowTrainer
from omegaconf import OmegaConf
import numpy as np
import torch
from tqdm import tqdm
from data import FeaturesTrainDataset, FeaturesDatasetOOD

In [2]:
cfg_path = "/workspaces/ood/fastflow/config.yaml"
cfg = OmegaConf.load(cfg_path)

In [3]:
train_size = 100
train_dataset = FeaturesTrainDataset(
    "/workspaces/ood/data/feature_maps/cifar10train_wide_resnet50_layer2_layer3_layer4_1000_img_per_class.npy",
    ["layer2", "layer3", "layer4"],
    [(train_size, 512, 28, 28), (train_size, 1024, 14, 14), (train_size, 2048, 7, 7)],
)

In [4]:
test_dataset = FeaturesDatasetOOD(
    "/workspaces/ood/data/feature_maps/cifar10test_wide_resnet50_layer2_layer3_layer4_100_img_per_class.npy",
    "/workspaces/ood/data/feature_maps/svhntest_wide_resnet50_layer2_layer3_layer4_100_img_per_class.npy",
    ["layer2", "layer3", "layer4"],
    [(1000, 512, 28, 28), (1000, 1024, 14, 14), (1000, 2048, 7, 7)],
)

In [5]:
train_dataloader = DataLoader(
    train_dataset, batch_size=cfg.trainer.batch_size, shuffle=True
)
test_dataloader = DataLoader(
    test_dataset, batch_size=cfg.trainer.batch_size, shuffle=False
)

In [6]:
trainer = FastflowTrainer(cfg)

trainer.train(train_dataloader, test_dataloader)


CNF coder: 512
CNF coder: 1024
CNF coder: 2048


  0%|          | 0/7 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py:64: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:174.)
  return default_collate([torch.as_tensor(b) for b in batch])
 14%|█▍        | 1/7 [00:03<00:21,  3.57s/it]

33.387866139411926


 29%|██▊       | 2/7 [00:04<00:13,  2.69s/it]

35.66893827915192


...

nan


 86%|████████▌ | 6/7 [00:06<00:01,  1.13s/it]

nan


100%|██████████| 7/7 [00:07<00:00,  1.02s/it]

nan
nan


In [7]:
torch.save(trainer.model.state_dict(), f'debug_model.pth')

In [8]:
trainer.model.state_dict()

OrderedDict([('decoders.0.module_list.0.perm',
              tensor([ 53, 407, 451,  85, 176, 333,  71, 237, 100, 228, 426,  82, 444, 318,
                      389, 329,  84, 210, 255, 475, 207, 369, 382, 233, 156,  50, 170, 427,
                      509, 500, 183, 239, 293, 150, 284, 159,  21, 337,  23, 483,  74, 256,
                      214,  87,   6,  56, 184, 194, 375, 398, 261, 189, 476, 363, 391, 371,
                      298, 312, 148, 235, 112, 373, 282, 116, 421, 105, 223, 166, 463, 459,
                      212, 457,  86,  67, 358, 161,  54,  36, 468, 411,  44,  37, 225, 292,
                      265,  72, 310,  52, 238, 339, 487, 157, 360, 188, 201, 121, 446, 243,
                      276, 124, 441, 334, 502, 200, 162,   5, 111, 370, 164,  73, 102, 174,
                      277, 278, 205, 218,  81,  89, 404, 181, 165, 172, 108,  34, 347, 115,
                      324, 226, 242, 134, 190, 294, 378, 387,  41, 122, 249, 381, 290, 366,
                      478,  66, 2

In [11]:
trainer = FastflowTrainer(cfg)
trainer.model.load_state_dict(torch.load('debug_model.pth'))

CNF coder: 512
CNF coder: 1024
CNF coder: 2048


<All keys matched successfully>

In [12]:
from model import FastflowModel, get_logp
def make_predictions(model, dataset, device):
    """
    returns logits for each samlple in dataset
    """
    model.eval()
    preds_in = []
    preds_out = []

    
    for emb, label in tqdm(dataset):
        avg_loss = 0
        for layer_idx, encoder_activations in enumerate(emb):
            encoder_activations = torch.unsqueeze(torch.tensor(encoder_activations).to(device), dim=0)
            (
                _,
                dim_feature_vector,
                im_height,
                im_width,
            ) = encoder_activations.size()
            decoder = model.decoders[layer_idx].to(device)
            p_u, log_jac_det = decoder(encoder_activations)
            print(log_jac_det, flush=True)
            return
            decoder_log_prob = get_logp(dim_feature_vector, p_u, log_jac_det)
            avg_loss += decoder_log_prob.sum().detach().cpu().numpy()
            print(p_u)
            return
        if label == 0:
            preds_in.append(avg_loss)
        elif label == 1:
            preds_out.append(avg_loss)
        else:
            raise ValueError
        print(preds_in, preds_out)
        return
            
    
    return preds_in, preds_out


preds = make_predictions(trainer.model, test_dataset, torch.device('cuda'))

  0%|          | 0/2000 [00:00<?, ?it/s]

tensor([1162990.3750], device='cuda:0', grad_fn=<AddBackward0>)


  0%|          | 0/2000 [00:00<?, ?it/s]


In [24]:
trainer.model

<All keys matched successfully>